# Практическая работа №6

# Выполнил студент группы ББМО-01-23 Бакин Д.И.

In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np

# Шаг 1: Загрузка данных и создание моделей
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(train_images, train_labels, epochs=2)
model1.save('mnist_model1.h5')
# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=2)
model2.save('mnist_model2.h5')



Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8798 - loss: 0.4288
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9645 - loss: 0.1241


Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9112 - loss: 0.3038
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9842 - loss: 0.0543


In [12]:
# Шаг 2: Реализация FGSM атаки
# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbed_image = image + epsilon * np.sign(gradient)
    perturbed_image = np.clip(perturbed_image, 0, 1)  # Ограничение значений в диапазоне [0, 1]
    return perturbed_image

# Генерация противоречивых примеров
def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape(1, 28, 28, 1), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape(1, -1), dtype=tf.float32)

        # Вычисление градиента
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)

        gradient = tape.gradient(loss, image)
        adv_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adv_image.reshape(28, 28))

    return np.array(adversarial_images)

# Создание противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)



In [13]:
# Шаг 3: Оценка результатов
# Оценка первой модели на противоречивых примерах
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels, verbose=0)
print(f'Accuracy of model1 on adversarial examples: {acc1:.4f}')

# Оценка второй модели на противоречивых примерах первой модели
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels, verbose=0)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2:.4f}')



Accuracy of model1 on adversarial examples: 0.1360
Accuracy of model2 on adversarial examples from model1: 0.9588


In [14]:
# Шаг 4: Дополнительное задание
# Создание противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2, test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels, verbose=0)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3:.4f}')

# Оценка второй модели на собственных противоречивых примерах
# Оценка второй модели на собственных противоречивых примерах
adversarial_images_model2 = adversarial_images_model2.reshape(-1, 28, 28, 1)
adversarial_images_model2 = adversarial_images_model2.astype('float32')

loss4, acc4 = model2.evaluate(adversarial_images_model2, test_labels, verbose=0)
print(f'Accuracy of model2 on its own adversarial examples: {acc4:.4f}')


Accuracy of model1 on adversarial examples from model2: 0.9258
Accuracy of model2 on its own adversarial examples: 0.6750


## Вывод

В ходе выполнения работы была исследована концепция атаки по переносу (Transfer Attack) на модели искусственного интеллекта (ИИ). Атака по переносу позволяет использовать противоречивые примеры, созданные для одной модели, для атаки на другую модель, даже если они обучены на разных архитектурах. Это явление имеет важное значение для безопасности ИИ-систем, поскольку показывает, что уязвимости могут быть общими для разных моделей.
Работа включала следующие этапы:

 1 Создание и обучение моделей: Были созданы две модели на основе датасета MNIST: полносвязная нейронная сеть и сверточная нейронная сеть (CNN). Обе модели были обучены на одном и том же наборе данных, что позволило провести анализ их уязвимости.

 2 Реализация атаки FGSM: Для первой модели была реализована атака FGSM (Fast Gradient Sign Method), с помощью которой были сгенерированы противоречивые примеры. Эти примеры затем использовались для атаки на вторую модель, что позволило изучить переносимость атаки.

 3 Оценка переносимости атаки: Оценка точности моделей на противоречивых примерах показала, что противоречивые примеры, созданные для первой модели, снижают точность второй модели. Это подтверждает, что атаки могут быть перенесены с одной модели на другую, даже если архитектуры моделей различаются.

 4 Дополнительное задание: В рамках дополнительного задания были сгенерированы противоречивые примеры для второй модели и проверены на первой. Результаты подтвердили, что обе модели уязвимы к атакам, даже если они имеют разные архитектуры.

В результате работы было продемонстрировано, что атаки могут переноситься между моделями с различными архитектурами, что является серьёзной угрозой для безопасности ИИ-систем. Для повышения устойчивости моделей к подобным атакам важно применять методы защиты, такие как обучение с учётом противоречивых примеров (adversarial training).
Таким образом, задача по переносу атак подчеркнула важность разработки надёжных и устойчивых моделей ИИ, способных противостоять подобным угрозам.